# Modelo LDA

Se utilizó Latent Dirichlet Allocation(LDA) haciendo uso de la implementación del paquete Gensim. Esta algoritmo realiza una reducción de dimensionalidad, a partir del cual se realiza un agrupamientos de palabras que pertenecen al mismo tópico. Los tópicos producidos por el modelo son *clusters* de palabras similares.


In [1]:
%matplotlib inline

import spacy
import pickle
from utils import read_file, get_words_and_lemma, create_counter_word_vector
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
#parametrización
TEXT_FILE = 'lavoztextodump.txt'	# Text to be processed
ENCODING = 'utf8'
MIN_FREQUENCY = 80	# Min word frequency to be considered
MIN_WORDS_COUNT_IN_SENT=10
POS_TAG= ['PROPN', 'ADJ', 'NOUN', 'VERB', 'ADV']
NUM_TOPICS=30

In [3]:
nlp = spacy.load('es_core_news_sm', disable=['parser', 'ner'])
nlp.max_length = 34886712

dataset = read_file(TEXT_FILE, ENCODING)
#doc = nlp(dataset)

filename = "trained/doc_npl_la_voz.pickle"
#fileObj = open(filename, 'wb')
#pickle.dump(doc, fileObj)
#fileObj.close()

with open(filename, 'rb') as f:
    doc = pickle.load(f)

## Pre procesamiento

Se realizan los siguientes procesamientos:
1. Eliminación de *stop words*
2. Lematización utilizando Spacy
3. Eliminación de las palabras menos frecuentes
4. Identificación de biagramas y trigramas: los bigramas son dos palabras que aparecen juntas con frecuencia en el documento y los trigramas son tres palabras que aparecen juntas con frecuencia en el documento. Para hacer esto se uso Phrases de Gensim.
5. Eliminación de palabras no alfanuméricas
6. Solo se consideran palabras cuyo *part-of-speech* son un nombre propio, un adjetivo, un adverbio, un sustantivo o un verbo. [Ver *Universal POS tags*](https://universaldependencies.org/docs/u/pos/)

In [4]:
#identificación de las frecuencias de las palabras
words, words_lemma = get_words_and_lemma(doc, MIN_WORDS_COUNT_IN_SENT, POS_TAG)

counts = create_counter_word_vector(words_lemma)

sents = [sent for sent in doc.sents if len(sent) > MIN_WORDS_COUNT_IN_SENT] #Se eliminan oraciones con menos de 10 palabras

#se obtiene un array con las oraciones (en formato texto), cuyas palabras han pasado por un proceso de lematización y filtrado
doc_list = []
for indexsent, sent in enumerate(sents):
    resultArray = []
    for word in sent:
        if word.is_alpha and not word.is_stop and word.pos_ in POS_TAG and counts[word.lemma_] > MIN_FREQUENCY:
            resultArray.append(word.lemma_)
    if len(resultArray) > 0:
        doc_list.append(resultArray)


#Se convierte cada oración en una lista de palabras, eliminando las puntuaciones y caracteres innecesarios
#para esto usamos simple_preprocess () de Gensim
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(doc_list))

#print(data_words[:1])

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[3]]])

# Define functions for bigrams and trigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

# Form Bigrams
data_lemmatized = make_trigrams(data_words_bigrams)

#Filtrado de oraciones con minima cantidad de palabras
#data_lemmatized = []
#for temp in data_lemmatized_temp:
#    if len(temp) > 5:
#        data_lemmatized.append(temp)
#print(data_lemmatized[:1])


## Crear diccionario y corpus

Las dos principales entradas para un modelo LDA son el diccionario (id2word) y el corpus.

Gensim va a crear identificadores únicos para cada palabra en el documento. Este identificador es usado como entrada por el modelo LDA.

In [5]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('martinez', 1), ('pareja', 1), ('sostener', 1)]]

## Construir modelo LDA

Entrenamos el modelo LDA. Además de pasar el corpus y diccionario, se define el número de tópicos a encontrar.


In [6]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=NUM_TOPICS,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

filename = "trained/doc_lda_model_result.pickle"
fileObj = open(filename, 'wb')
pickle.dump(doc, fileObj)
fileObj.close()

#with open(filename, 'rb') as f:
#    lda_model = pickle.load(f)

In [7]:
# Compute Perplexity
#print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
#coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
#coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)

## Visualización de tópicos y palabras claves

Se usa pyLDAvis para mostrar los tópicos y las palabras claves. Esta es una herramienta gráfico interactiva.

### Interpretación del gráfico:

Cada burbuja en el gráfico del lado izquierdo representa un tópico. Cuanto más grande es la burbuja, más frecuente es ese tópico. A la derecha del gráfico aparecen las palabras claves del tópico.

Para nuestro caso podemos observar un modelo con demasiados tópicos, ya que vemos muchos tópicos superpuestos y burbujas de tamaño pequeño agrupadas en una región del gráfico.


In [8]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.103544  0.444478       1        1  10.529102
24     0.117692 -0.107548       2        1   6.242111
19     0.059195  0.061216       3        1   6.074005
6      0.048912  0.045282       4        1   5.467170
29     0.041407  0.056705       5        1   5.220577
28     0.058108 -0.024080       6        1   4.355187
17     0.083807 -0.026948       7        1   4.309258
26     0.042860 -0.003449       8        1   4.236345
4      0.090845 -0.051827       9        1   4.226243
1      0.022205 -0.008729      10        1   4.121759
14     0.023770 -0.012040      11        1   3.744866
2      0.052408 -0.022681      12        1   3.711330
0      0.038228 -0.025385      13        1   3.539483
8      0.056798 -0.038584      14        1   3.483498
21     0.060082 -0.042889      15        1   3.452510
11     0.069227 -0.052502      16        1   3.194378
25     0.035900 -0.022877      17        1   3.083238
27    -0.004200 -0.014684      18        1   3.080163
5      0.050875 -0.034999      19        1   3.012639
9     -0.007359 -0.016400      20        1   2.756533
13     0.008468 -0.020981      21        1   2.569333
18     0.030204 -0.030663      22        1   2.448277
20     0.046377 -0.032911      23        1   2.098259
7      0.006249 -0.037044      24        1   1.337148
3     -0.029256 -0.022466      25        1   1.232434
22    -0.114495 -0.004442      26        1   0.991509
15    -0.066032 -0.011464      27        1   0.966992
23    -0.233201  0.012532      28        1   0.371308
12    -0.346308  0.022691      29        1   0.072173
16    -0.346308  0.022691      30        1   0.072173, topic_info=          Term          Freq         Total Category  logprob  loglift
3          ano  16128.000000  16128.000000  Default   30.000  30.0000
88     cordoba  11201.000000  11201.000000  Default   29.000  29.0000
72         ley   6655.000000   6655.000000  Default   28.000  28.0000
91    gobierno   9043.000000   9043.000000  Default   27.000  27.0000
242       caso   5685.000000   5685.000000  Default   26.000  26.0000
..         ...           ...           ...      ...      ...      ...
25        vida      0.147706   2567.024726  Topic30   -7.335  -2.5292
26      ayudar      0.147706    350.133030  Topic30   -7.335  -0.5370
27   compartir      0.147706    179.409239  Topic30   -7.335   0.1317
28      dar_el      0.147706    340.027542  Topic30   -7.335  -0.5077
29      muerte      0.147706    951.362030  Topic30   -7.335  -1.5366

[1298 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
94        5  0.995711      abrir
156      19  0.998662     accion
208      13  0.994534    aceptar
179       3  0.986657     acerca
170       4  0.996533       acto
...     ...       ...        ...
103      11  0.994472      villa
187       3  0.994363  vinculado
200      21  0.981641      vista
19       13  0.997896      vivir
93        4  0.998088     volver

[237 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 25, 20, 7, 30, 29, 18, 27, 5, 2, 15, 3, 1, 9, 22, 12, 26, 28, 6, 10, 14, 19, 21, 8, 4, 23, 16, 24, 13, 17])

In [9]:
for index, topic in lda_model.show_topics(num_topics=30, formatted=False, num_words= 30):
    print('Topic: {} \nWords: {}'.format(index, [w[0] for w in topic]))

Topic: 0 
Words: ['persona', 'asegurar', 'vivir', 'cambiar', 'aceptar', 'preguntar', 'comprar', 'rodriguez', 'carta', 'realidad', 'empresa', 'pesos', 'presentar', 'autoridad', 'libertad', 'nacional', 'provincia', 'nacion', 'privado', 'religioso', 'tener', 'derecho', 'condena', 'servicio', 'auto', 'tucuman', 'investigar', 'peronismo', 'muerto', 'policia']
Topic: 1 
Words: ['social', 'informacion', 'chico', 'responsable', 'ciudadano', 'posicion', 'crecer', 'base', 'respetar', 'democratico', 'consenso', 'dialogo', 'presidente', 'juicio', 'pesos', 'sector', 'capital', 'empresa', 'regimen', 'civil', 'millon', 'innviron', 'tema', 'publico', 'militar', 'sociedad', 'pais', 'politico', 'nacional', 'aca']
Topic: 2 
Words: ['tomar', 'decir', 'agua', 'medida', 'discusion', 'madre', 'sumar', 'completo', 'sabado', 'empresa', 'pais', 'nacional', 'innviron', 'aca', 'maquina', 'luz', 'chofer', 'foto', 'seguro', 'procedimiento', 'iglesia', 'pleno', 'referir', 'pie', 'traer', 'instalar', 'chile', 'superf